#Imports, Installations, Dataset Download, and Sign-in

In [ ]:
!pip install opendatasets

In [ ]:
!pip install -U -q datasets transformers[torch] evaluate timm albumentations accelerate torchmetrics

In [ ]:
pip install lightning

In [ ]:
!pip install huggingface_hub

In [ ]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import shutil
from google.colab.patches import cv2_imshow
import opendatasets as od
import torch
import torchvision
from pathlib import Path
from huggingface_hub import hf_hub_download
from tqdm.auto import tqdm
from PIL import Image as im
import albumentations as A
import lightning as l
import random
import requests
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from datasets import load_dataset, Image, DatasetDict, Dataset


In [ ]:
!huggingface-cli login

In [ ]:
od.download("https://www.kaggle.com/datasets/lingfengzhang/pediatric-polymicrogyria-mri-dataset")

#Balanced Dataset Creation

In [ ]:
image_folders = []
for i in os.listdir("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021"):
  for j in os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021", i)):
    image_folders.append(len(os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021", i, j))))

In [ ]:
sum(image_folders)

In [ ]:
sum(image_folders)

In [ ]:
PPMR_image_paths = []
for i in os.listdir("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGstudycaseslabelled"):
  for j in os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGstudycaseslabelled", i)):
    PPMR_image_paths+=[os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGstudycaseslabelled",i,j,k) for k in os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGstudycaseslabelled", i, j))]

In [ ]:
len(PPMR_image_paths)

In [ ]:
control_image_paths = []
for i in os.listdir("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021"):
  for j in os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021", i)):
    control_image_paths+=[os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021",i,j,k) for k in os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021", i, j))]

In [ ]:
len(control_image_paths)

In [ ]:
control_image_paths[0]

In [ ]:
random.shuffle(control_image_paths)
random.shuffle(PPMR_image_paths)

In [ ]:
control_images_train_test_split = int(4517*0.8)
PPMR_images_train_test_split = int(4517*0.8)

In [ ]:
train_control_images = control_image_paths[:control_images_train_test_split]
test_control_images = control_image_paths[control_images_train_test_split:4517]
train_PPMR_images = PPMR_image_paths[:PPMR_images_train_test_split]
test_PPMR_images = PPMR_image_paths[PPMR_images_train_test_split:4517]

In [ ]:
len(train_control_images), len(test_control_images), len(train_PPMR_images), len(test_PPMR_images)

In [ ]:
!mkdir folder
!mkdir folder/train
!mkdir folder/test
!mkdir folder/train/ppmr
!mkdir folder/train/control
!mkdir folder/test/ppmr
!mkdir folder/test/control

In [ ]:
for i in train_control_images:
  shutil.copy(i, "/content/folder/train/control/")
for i in train_PPMR_images:
  shutil.copy(i, "/content/folder/train/ppmr/")
for i in test_control_images:
  shutil.copy(i, "/content/folder/test/control/")
for i in test_PPMR_images:
  shutil.copy(i, "/content/folder/test/ppmr/")

In [ ]:
len(os.listdir("/content/folder/train/control")), len(os.listdir("/content/folder/train/ppmr")), len(os.listdir("/content/folder/test/control")), len(os.listdir("/content/folder/test/ppmr/"))

In [ ]:
dataset = load_dataset("imagefolder", data_dir="/content/folder")

In [ ]:
dataset.push_to_hub("sha000/ppmr-large-balanced")

# Full Unbalanced Dataset creation

In [ ]:
image_folders = []
for i in os.listdir("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021"):
  for j in os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021", i)):
    image_folders.append(len(os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021", i, j))))

In [ ]:
sum(image_folders)

In [ ]:
sum(image_folders)

In [ ]:
PPMR_image_paths = []
for i in os.listdir("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGstudycaseslabelled"):
  for j in os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGstudycaseslabelled", i)):
    PPMR_image_paths+=[os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGstudycaseslabelled",i,j,k) for k in os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGstudycaseslabelled", i, j))]

In [ ]:
len(PPMR_image_paths)

In [ ]:
control_image_paths = []
for i in os.listdir("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021"):
  for j in os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021", i)):
    control_image_paths+=[os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021",i,j,k) for k in os.listdir(os.path.join("/content/pediatric-polymicrogyria-mri-dataset/PPMR/PMGControlsEditedDec2021", i, j))]

In [ ]:
len(control_image_paths)

In [ ]:
control_image_paths[0]

In [ ]:
random.shuffle(control_image_paths)
random.shuffle(PPMR_image_paths)

In [ ]:
control_images_train_test_split = int(len(control_image_paths)*0.8)
PPMR_images_train_test_split = int(PPMR_image_paths*0.8)

In [ ]:
train_control_images = control_image_paths[:control_images_train_test_split]
test_control_images = control_image_paths[control_images_train_test_split:]
train_PPMR_images = PPMR_image_paths[:PPMR_images_train_test_split]
test_PPMR_images = PPMR_image_paths[PPMR_images_train_test_split:]

In [ ]:
len(train_control_images), len(test_control_images), len(train_PPMR_images), len(test_PPMR_images)

In [ ]:
!mkdir folder
!mkdir folder/train
!mkdir folder/test
!mkdir folder/train/ppmr
!mkdir folder/train/control
!mkdir folder/test/ppmr
!mkdir folder/test/control

In [ ]:
for i in train_control_images:
  shutil.copy(i, "/content/folder/train/control/")
for i in train_PPMR_images:
  shutil.copy(i, "/content/folder/train/ppmr/")
for i in test_control_images:
  shutil.copy(i, "/content/folder/test/control/")
for i in test_PPMR_images:
  shutil.copy(i, "/content/folder/test/ppmr/")

In [ ]:
len(os.listdir("/content/folder/train/control")), len(os.listdir("/content/folder/train/ppmr")), len(os.listdir("/content/folder/test/control")), len(os.listdir("/content/folder/test/ppmr/"))

In [ ]:
dataset = load_dataset("imagefolder", data_dir="/content/folder")

In [ ]:
dataset.push_to_hub("sha000/ppmr-full-unbalanced")

#Google ViT Training

In [ ]:
dataset = load_dataset("sha000/ppmr-large-balanced")

In [ ]:
test_dataset = dataset["test"]
train_dataset = dataset["train"]

In [ ]:
train_dataset.features

In [ ]:
train_dataset[0]

In [ ]:
id2label = {id:label for id, label in enumerate(train_dataset.features['label'].names)}
label2id = {label:id for id,label in id2label.items()}
id2label

In [ ]:
id2label[train_dataset[0]['label']]


In [ ]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()


In [ ]:
from torchvision.transforms import Resize,ToTensor, Compose
_transforms = Compose([Resize((224,224)), ToTensor()])

def transforms(examples):

	examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
	del examples["image"]
	return examples

train_transformed = train_dataset.with_transform(transforms)

In [ ]:
dataset_transformed = dataset.with_transform(transforms)

In [ ]:
import evaluate
def compute_metrics(eval_pred):
   metric1 = evaluate.load("accuracy")
   metric2 = evaluate.load("f1")
   metric3 = evaluate.load("precision")
   metric4 = evaluate.load("recall")
   predictions, labels = eval_pred
   predictions = np.argmax(predictions, axis=1)
   accuracy = metric1.compute(predictions=predictions,
                              references=labels)
   f1 = metric2.compute(predictions=predictions,
                        references=labels, average="weighted")
   precision = metric3.compute(predictions=predictions,
                               references=labels, average="weighted")
   recall = metric4.compute(predictions=predictions,
                            references=labels, average="weighted")
   return {"precision":precision["precision"],"recall":recall["recall"],"accuracy":accuracy["accuracy"],"f1":f1["f1"]}

In [ ]:
from transformers import AutoModelForImageClassification
model = AutoModelForImageClassification.from_pretrained(
   "google/vit-base-patch16-224-in21k",
   num_labels=2,
   id2label=id2label,
   label2id=label2id,
   ignore_mismatched_sizes=True
)



In [ ]:
dataset_transformed["train"][0]

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
	output_dir="./ppmr_classifier",
	remove_unused_columns=False,
	evaluation_strategy="epoch",
	save_strategy="epoch",
	learning_rate=5e-5,
	per_device_train_batch_size=16,
	gradient_accumulation_steps=4,
	per_device_eval_batch_size=16,
	num_train_epochs=8,
	warmup_ratio=0.1,
	logging_steps=10,
	load_best_model_at_end=True,
	report_to="tensorboard",
	metric_for_best_model="accuracy"
)

trainer = Trainer(
	model=model,
	args=training_args,
	data_collator=data_collator,
	train_dataset=dataset_transformed["train"],
	eval_dataset=dataset_transformed["train"],
	tokenizer=processor,
	compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

#Facebook DeiT Training

In [ ]:
dataset = load_dataset("sha000/ppmr-large-balanced")

In [ ]:
test_dataset = dataset["test"]
train_dataset = dataset["train"]

In [ ]:
train_dataset.features

In [ ]:
train_dataset[0]

In [ ]:
id2label = {id:label for id, label in enumerate(train_dataset.features['label'].names)}
label2id = {label:id for id,label in id2label.items()}
id2label

In [ ]:
id2label[train_dataset[0]['label']]


In [ ]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("facebook/deit-base-patch16-224")

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()


In [ ]:
from torchvision.transforms import Resize,ToTensor, Compose
_transforms = Compose([Resize((224,224)), ToTensor()])

def transforms(examples):

	examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
	del examples["image"]
	return examples

train_transformed = train_dataset.with_transform(transforms)

In [ ]:
dataset_transformed = dataset.with_transform(transforms)

In [ ]:
import evaluate
def compute_metrics(eval_pred):
   metric1 = evaluate.load("accuracy")
   metric2 = evaluate.load("f1")
   metric3 = evaluate.load("precision")
   metric4 = evaluate.load("recall")
   predictions, labels = eval_pred
   predictions = np.argmax(predictions, axis=1)
   accuracy = metric1.compute(predictions=predictions,
                              references=labels)
   f1 = metric2.compute(predictions=predictions,
                        references=labels, average="weighted")
   precision = metric3.compute(predictions=predictions,
                               references=labels, average="weighted")
   recall = metric4.compute(predictions=predictions,
                            references=labels, average="weighted")
   return {"precision":precision["precision"],"recall":recall["recall"],"accuracy":accuracy["accuracy"],"f1":f1["f1"]}

In [ ]:
from transformers import AutoModelForImageClassification
model = AutoModelForImageClassification.from_pretrained(
   "facebook/deit-base-patch16-224",
   num_labels=2,
   id2label=id2label,
   label2id=label2id,
   ignore_mismatched_sizes=True
)



In [ ]:
dataset_transformed["train"][0]

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
	output_dir="./ppmr_classifier",
	remove_unused_columns=False,
	evaluation_strategy="epoch",
	save_strategy="epoch",
	learning_rate=5e-5,
	per_device_train_batch_size=16,
	gradient_accumulation_steps=4,
	per_device_eval_batch_size=16,
	num_train_epochs=8,
	warmup_ratio=0.1,
	logging_steps=10,
	load_best_model_at_end=True,
	report_to="tensorboard",
	metric_for_best_model="accuracy"
)

trainer = Trainer(
	model=model,
	args=training_args,
	data_collator=data_collator,
	train_dataset=dataset_transformed["train"],
	eval_dataset=dataset_transformed["train"],
	tokenizer=processor,
	compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

#Microsoft BEiT Training

In [ ]:
dataset = load_dataset("sha000/ppmr-large-balanced")

In [ ]:
test_dataset = dataset["test"]
train_dataset = dataset["train"]

In [ ]:
train_dataset.features

In [ ]:
train_dataset[0]

In [ ]:
id2label = {id:label for id, label in enumerate(train_dataset.features['label'].names)}
label2id = {label:id for id,label in id2label.items()}
id2label

In [ ]:
id2label[train_dataset[0]['label']]


In [ ]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("microsoft/beit-base-patch16-224-pt22k-ft22k")

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()


In [ ]:
from torchvision.transforms import Resize,ToTensor, Compose
_transforms = Compose([Resize((224,224)), ToTensor()])

def transforms(examples):

	examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
	del examples["image"]
	return examples

train_transformed = train_dataset.with_transform(transforms)

In [ ]:
dataset_transformed = dataset.with_transform(transforms)

In [ ]:
import evaluate
def compute_metrics(eval_pred):
   metric1 = evaluate.load("accuracy")
   metric2 = evaluate.load("f1")
   metric3 = evaluate.load("precision")
   metric4 = evaluate.load("recall")
   predictions, labels = eval_pred
   predictions = np.argmax(predictions, axis=1)
   accuracy = metric1.compute(predictions=predictions,
                              references=labels)
   f1 = metric2.compute(predictions=predictions,
                        references=labels, average="weighted")
   precision = metric3.compute(predictions=predictions,
                               references=labels, average="weighted")
   recall = metric4.compute(predictions=predictions,
                            references=labels, average="weighted")
   return {"precision":precision["precision"],"recall":recall["recall"],"accuracy":accuracy["accuracy"],"f1":f1["f1"]}

In [ ]:
from transformers import AutoModelForImageClassification
model = AutoModelForImageClassification.from_pretrained(
   "microsoft/beit-base-patch16-224-pt22k-ft22k",
   num_labels=2,
   id2label=id2label,
   label2id=label2id,
   ignore_mismatched_sizes=True
)



In [ ]:
dataset_transformed["train"][0]

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
	output_dir="./ppmr_classifier",
	remove_unused_columns=False,
	evaluation_strategy="epoch",
	save_strategy="epoch",
	learning_rate=5e-5,
	per_device_train_batch_size=16,
	gradient_accumulation_steps=4,
	per_device_eval_batch_size=16,
	num_train_epochs=8,
	warmup_ratio=0.1,
	logging_steps=10,
	load_best_model_at_end=True,
	report_to="tensorboard",
	metric_for_best_model="accuracy"
)

trainer = Trainer(
	model=model,
	args=training_args,
	data_collator=data_collator,
	train_dataset=dataset_transformed["train"],
	eval_dataset=dataset_transformed["train"],
	tokenizer=processor,
	compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()